In [121]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os
from tqdm import tqdm_notebook as tqdm

In [144]:
res_stop = pd.read_csv('Videoanalitik.csv', sep = ';', encoding='utf-8')

In [145]:
res_stop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5173 entries, 0 to 5172
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Дата                        5173 non-null   object 
 1   ТС                          5173 non-null   object 
 2   Регистратор                 5015 non-null   float64
 3   Тип                         5173 non-null   object 
 4   Город                       5173 non-null   object 
 5   Перевозчик                  5173 non-null   object 
 6   Маршрут                     5173 non-null   int64  
 7   Вариант маршрута            5173 non-null   object 
 8   Ссылка на маршрут (Яндекс)  5173 non-null   object 
 9   route_id                    5173 non-null   int64  
 10  trip_id                     5173 non-null   int64  
 11  ID рейса видеоаналитики     5173 non-null   int64  
 12  Выход на маршрут            5173 non-null   object 
 13  Завершение машрута          5173 

In [151]:
res_stop['Открытие двери']=res_stop['Открытие двери'].fillna(0)
res_stop['time']=pd.to_datetime(res_stop['Открытие двери'])
res_stop['hour']=res_stop['time'].dt.hour
res_stop['min']=res_stop['time'].dt.minute
res_stop['sec']=res_stop['time'].dt.second
res_stop['time']=(res_stop['hour'].astype(int))*3600+(res_stop['min'].astype(int))*60+(res_stop['sec'].astype(int))

In [152]:
res_stop['Закрытие двери']=res_stop['Закрытие двери'].fillna(0)
res_stop['time2']=pd.to_datetime(res_stop['Закрытие двери'])
res_stop['hour']=res_stop['time2'].dt.hour
res_stop['min']=res_stop['time2'].dt.minute
res_stop['sec']=res_stop['time2'].dt.second
res_stop['time2']=(res_stop['hour'].astype(int))*3600+(res_stop['min'].astype(int))*60+(res_stop['sec'].astype(int))

In [153]:
res_stop2=res_stop.groupby(['Дата','ТС','Регистратор','Тип','Город','Перевозчик','Маршрут','Вариант маршрута','trip_id','ID рейса видеоаналитики','Выход на маршрут','Завершение машрута','# остановки в трассе','Остановка','# остановки в поездке']).agg({'Вошло': np.sum,'Вышло': np.sum,'time': np.min,'time2': np.max}).reset_index()

In [154]:
res_stop2.head()

,Дата,ТС,Регистратор,Тип,Город,Перевозчик,Маршрут,Вариант маршрута,trip_id,ID рейса видеоаналитики,Выход на маршрут,Завершение машрута,# остановки в трассе,Остановка,# остановки в поездке,Вошло,Вышло,time,time2
0,11.09.2020,ТрамСам.11(2209),21694.0,Трамвай,Самара,ТТУ,3,Пионерская улица—Юнгородок,2193179252,8568,7:37:42,9:00:21,1.0,Пионерская улица,1.0,0.0,13.0,27467,27482
1,11.09.2020,ТрамСам.11(2209),21694.0,Трамвай,Самара,ТТУ,3,Пионерская улица—Юнгородок,2193179252,8568,7:37:42,9:00:21,2.0,Чапаевская улица,2.0,1.0,2.0,27551,27578
2,11.09.2020,ТрамСам.11(2209),21694.0,Трамвай,Самара,ТТУ,3,Пионерская улица—Юнгородок,2193179252,8568,7:37:42,9:00:21,3.0,Улица Фрунзе,3.0,2.0,0.0,27615,27647
3,11.09.2020,ТрамСам.11(2209),21694.0,Трамвай,Самара,ТТУ,3,Пионерская улица—Юнгородок,2193179252,8568,7:37:42,9:00:21,4.0,Ленинградская улица,4.0,1.0,0.0,27744,27753
4,11.09.2020,ТрамСам.11(2209),21694.0,Трамвай,Самара,ТТУ,3,Пионерская улица—Юнгородок,2193179252,8568,7:37:42,9:00:21,5.0,Филармония,5.0,0.0,0.0,27865,27874


In [155]:
res_stop2.to_csv('for_analyze.csv')

In [124]:
Analyze_val = pd.read_csv('Analyze_val.csv', sep = ';', encoding='utf-8')
Analyze_val['time2']=Analyze_val['time']

In [125]:
Analyze_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2465 entries, 0 to 2464
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Дата и время поездки  2465 non-null   object 
 1   Терминал              2465 non-null   int64  
 2   Номер билета          2465 non-null   int64  
 3   Номер кристалла       1647 non-null   object 
 4   TCAT                  2465 non-null   object 
 5   NUM_TRIP              2465 non-null   int64  
 6   Код Маршрута          2465 non-null   object 
 7   Табельный номер       2465 non-null   int64  
 8   Код предприятия       2465 non-null   int64  
 9   mr_id                 2465 non-null   int64  
 10  route_id              2465 non-null   int64  
 11  trip_id               2465 non-null   int64  
 12  time                  2465 non-null   int64  
 13  start                 2465 non-null   int64  
 14  finish                2465 non-null   int64  
 15  tabelnum             

In [49]:
Analyze_val=Analyze_val.groupby(['TCAT','NUM_TRIP','trip_id','tabelnum','ssId','ss_name','ss_lat','ss_long','rc_orderby']).agg({'c': np.sum,'time': np.min,'time2': np.max}).reset_index()

In [50]:
Analyze_val.head()

,TCAT,NUM_TRIP,trip_id,tabelnum,ssId,ss_name,ss_lat,ss_long,rc_orderby,c,time,time2
0,БАНК,0,2193179252,1733.0,97,Авиагородок,53.209402,50.281997,35.0,34,62934,76429
1,БАНК,0,2193179252,2832.0,97,Авиагородок,53.209402,50.281997,35.0,5,33136,35177
2,БАНК,0,2193179256,1733.0,330,ул.Волгина,53.191537,50.184721,15.0,2,53886,53903
3,БАНК,0,2193179256,1733.0,421,ул.Арцыбушевская,53.201103,50.128917,26.0,2,55242,55250
4,БАНК,0,2193179256,1733.0,653,ул.Гагарина,53.201932,50.229975,8.0,1,53123,53123


In [62]:
res_stop2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4975 entries, 0 to 4974
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Дата                     4975 non-null   object 
 1   ТС                       4975 non-null   object 
 2   Регистратор              4975 non-null   float64
 3   Тип                      4975 non-null   object 
 4   Город                    4975 non-null   object 
 5   Перевозчик               4975 non-null   object 
 6   Маршрут                  4975 non-null   object 
 7   Вариант маршрута         4975 non-null   object 
 8   trip_id                  4975 non-null   int64  
 9   ID рейса видеоаналитики  4975 non-null   int64  
 10  Выход на маршрут         4975 non-null   object 
 11  Завершение машрута       4975 non-null   object 
 12  # остановки в трассе     4975 non-null   float64
 13  Остановка                4975 non-null   object 
 14  Широта                  

In [64]:
Analyze_val['trip_id'].unique()

array([2193179252, 2193179256, 2193179661, 2193179869, 2193179888,
       2193179889, 2193179919, 2193179920, 2193179658], dtype=int64)

In [65]:
res_stop2['trip_id'].unique()

array([2193179144, 2193179145, 2193179252, 2193179256, 2193179869,
       2193179658, 2193179661, 2193179846, 2193179888, 2193179889,
       2193179920, 2193179919], dtype=int64)

In [72]:
Analyze_val[(Analyze_val['trip_id']==2193179256)&(Analyze_val['time']>52790)&(Analyze_val['time']<53141)  ]

,TCAT,NUM_TRIP,trip_id,tabelnum,ssId,ss_name,ss_lat,ss_long,rc_orderby,c,time,time2
4,БАНК,0,2193179256,1733.0,653,ул.Гагарина,53.201932,50.229975,8.0,1,53123,53123
307,СОЦ,61,2193179256,1733.0,140,ул.Красных Коммунаров,53.203344,50.238175,7.0,1,53098,53098
624,СОЦ,81,2193179256,1733.0,140,ул.Красных Коммунаров,53.203344,50.238175,7.0,1,53093,53093


In [70]:
res_stop2[(res_stop2['trip_id']==2193179256)&(res_stop2['time']>52790)]

,Дата,ТС,Регистратор,Тип,Город,Перевозчик,Маршрут,Вариант маршрута,trip_id,ID рейса видеоаналитики,...,Долгота,# остановки в поездке,Вошло,Вышло,time,time2,ssId,ss_name,TCAT,tabelnum
1097,11.09.2020,ТрамСам.11(2209),21694.0,Трамвай,Самара,ТТУ,3,Юнгородок—Пионерская улица,2193179256,8573,...,"50,23815388",7.0,0.0,0.0,52990,53004,0,0,0,0.0
1098,11.09.2020,ТрамСам.11(2209),21694.0,Трамвай,Самара,ТТУ,3,Юнгородок—Пионерская улица,2193179256,8573,...,"50,23815425",7.0,3.0,1.0,52991,53003,0,0,0,0.0
1099,11.09.2020,ТрамСам.11(2209),21694.0,Трамвай,Самара,ТТУ,3,Юнгородок—Пионерская улица,2193179256,8573,...,"50,23815467",7.0,0.0,1.0,52990,53002,0,0,0,0.0
1100,11.09.2020,ТрамСам.11(2209),21694.0,Трамвай,Самара,ТТУ,3,Юнгородок—Пионерская улица,2193179256,8573,...,"50,239281",7.0,1.0,0.0,52932,52954,0,0,0,0.0
1101,11.09.2020,ТрамСам.11(2209),21694.0,Трамвай,Самара,ТТУ,3,Юнгородок—Пионерская улица,2193179256,8573,...,"50,22984365",9.0,1.0,0.0,53128,53141,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2453,11.09.2020,ТрамСам.3.2,21644.0,Трамвай,Самара,ТТУ,3,Юнгородок—Пионерская улица,2193179256,9228,...,"50,09176885",33.0,0.0,0.0,52791,52799,0,0,0,0.0
2455,11.09.2020,ТрамСам.3.2,21644.0,Трамвай,Самара,ТТУ,3,Юнгородок—Пионерская улица,2193179256,9228,...,"50,08999962",34.0,0.0,0.0,52851,52860,0,0,0,0.0
2456,11.09.2020,ТрамСам.3.2,21644.0,Трамвай,Самара,ТТУ,3,Юнгородок—Пионерская улица,2193179256,9228,...,"50,09000017",34.0,0.0,1.0,52851,52859,0,0,0,0.0
2457,11.09.2020,ТрамСам.3.2,21644.0,Трамвай,Самара,ТТУ,3,Юнгородок—Пионерская улица,2193179256,9228,...,"50,08925587",35.0,1.0,1.0,53003,53014,0,0,0,0.0


In [ ]:
videoanal=pd.read_csv('For_analyze.csv', sep = ',', encoding='utf-8')

In [135]:
valid=pd.read_csv('Analyze_val.csv', sep = ';', encoding='utf-8')

In [137]:
valid=valid.sort_values(by=['Код Маршрута','trip_id','time'])

In [138]:
valid.head()

,Дата и время поездки,Терминал,Номер билета,Номер кристалла,TCAT,NUM_TRIP,Код Маршрута,Табельный номер,Код предприятия,mr_id,...,start,finish,tabelnum,ssId,rl_id,ss_name,ss_lat,ss_long,c,rc_orderby
1856,11.09.2020 5:27,82843822,1,9E788824,СОЦ,73,"1,16103E+11",805,5,152,...,19644,68364,1127.0,703,44665,Микрорайон Крутые ключи,53.309470,50.309594,1,1.0
1857,11.09.2020 5:27,82843822,2,37DB4FE6,СОЦ,70,"1,16103E+11",805,5,152,...,19644,68364,1127.0,703,44665,Микрорайон Крутые ключи,53.309470,50.309594,1,1.0
1858,11.09.2020 5:27,82843822,3,NaN,БАНК,0,"1,16103E+11",805,5,152,...,19644,68364,1127.0,703,44665,Микрорайон Крутые ключи,53.309470,50.309594,1,1.0
1865,11.09.2020 5:27,82843822,4,47E69100,СОЦ,69,"1,16103E+11",805,5,152,...,19644,68364,1127.0,702,44665,ул.Мира,53.312815,50.307600,1,2.0
1882,11.09.2020 5:29,82843822,6,6339CF4A,КС,0,"1,16103E+11",805,5,152,...,19644,68364,1127.0,700,44665,Храм в честь Сретения Господня,53.317124,50.303654,1,3.0


In [139]:
valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2465 entries, 1856 to 1599
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Дата и время поездки  2465 non-null   object 
 1   Терминал              2465 non-null   int64  
 2   Номер билета          2465 non-null   int64  
 3   Номер кристалла       1647 non-null   object 
 4   TCAT                  2465 non-null   object 
 5   NUM_TRIP              2465 non-null   int64  
 6   Код Маршрута          2465 non-null   object 
 7   Табельный номер       2465 non-null   int64  
 8   Код предприятия       2465 non-null   int64  
 9   mr_id                 2465 non-null   int64  
 10  route_id              2465 non-null   int64  
 11  trip_id               2465 non-null   int64  
 12  time                  2465 non-null   int64  
 13  start                 2465 non-null   int64  
 14  finish                2465 non-null   int64  
 15  tabelnum          

In [140]:
k=1
res=valid
result = []
p=0
cod_marsh=0
for index, row in valid.iterrows():
    if row['Код Маршрута']!=cod_marsh:
        k=0
    if row['rc_orderby']<p:
        k=k+1
        p=row['rc_orderby']
    else:
        p=row['rc_orderby']
    cod_marsh=row['Код Маршрута']
    result.append(k)
valid['reis']=result

In [141]:
valid['reis']

1856    0
1857    0
1858    0
1865    0
1882    0
       ..
1473    6
1474    6
1475    6
1476    6
1599    6
Name: reis, Length: 2465, dtype: int64

In [142]:
valid_=pd.DataFrame(valid.groupby(['Терминал','trip_id','ssId','ss_name','reis','rc_orderby']).agg({'c': np.sum,'time':np.min})).reset_index()
valid_['time2']= valid_.groupby(['Терминал','trip_id','reis'])['time'].shift(-1).astype('Int64')

In [143]:
valid_.to_csv('for_analyze_val.csv')

In [119]:
videoanal=videoanal[['trip_id','ID рейса видеоаналитики','# остановки в трассе','# остановки в поездке','Остановка','Вошло','Вышло','time','time2']]
videoanal_=pd.DataFrame(videoanal.groupby(['trip_id','ID рейса видеоаналитики','# остановки в трассе','# остановки в поездке','Остановка']).agg({'Вошло': np.sum,'Вышло': np.sum,'time':np.min,'time2':np.max})).reset_index()
videoanal_['time_2']= videoanal_.groupby(['trip_id','ID рейса видеоаналитики'])['time'].shift(-1).astype('Int64')

In [120]:
valid_.to_csv('valid_itog.csv')
videoanal_.to_csv('videoanal_.csv')

In [106]:
k=0
res=videoanal_
result = []
result2 = []
result3 = []
result4 = []
stops=pd.DataFrame()
for index, row in videoanal_.iterrows():
    stops=valid_[(valid_['trip_id']==row['trip_id'])
                    &(valid_['time2']<=row['time2']+100)
                    &(valid_['time']>=row['time'])-100]
    if stops.empty:
        result.append(0)
        result2.append(0)
        result3.append(0)
        result4.append(0)
    else:
        stop_unic=stops['ssId'].values
        result.append(stop_unic[0])
        stop_unic2=stops['ss_name'].values
        result2.append(stop_unic2[0])
        stop_unic3=stops['c'].values
        result3.append(stop_unic3[0])
        #Analyze_val = Analyze_val[~Analyze_val.index.isin(stops[0:1].index)]
res['ssId']=result
res['ss_name']=result2
res['c']=result3

In [107]:
len(res[res['ssId']==0])

2123

In [108]:
len(res)

2247

In [109]:
res.to_csv('res.csv')